In [1]:
import pandas as pd
import numpy as np
import pygtrie as trie
from collections import defaultdict
from scipy.stats import entropy

In [2]:
df = pd.read_csv("output_suffix.txt", sep=' ', header=None, names=["word","cnt","err"])

In [3]:
m = {} # 存储每个候选词的词频
m2 = defaultdict(lambda: defaultdict(int)) # 存储每个候选词的前缀字集合
for index, row in df.iterrows():
    m[row['word']] = row['cnt']
    if len(row['word']) == 1: continue # 单字词没有前缀字，无需计算
    key = row['word'][1:]
    prefix = row['word'][:1]
    m2[key][prefix] += row['cnt']

### 计算聚合度，聚合度定义为当前串频次与其后缀串频次的比值

In [4]:
def poly(x):
    if len(x['word']) == 1 or x['word'][1:] not in m: return 1 # 单字聚合度为1
    return x['cnt']/m[x['word'][1:]]

### 计算灵活度，灵活度定义为前缀字集合的信息熵

In [5]:
def flex(x):
    if x['word'] not in m2: return 1
    s = pd.Series(m2[x['word']])
    return entropy(s / s.sum())

In [6]:
df['poly'] = df.apply(poly, axis=1)
df['flex'] = df.apply(flex, axis=1)
df['score'] = df.apply(lambda x: x['flex']*x['poly'], axis=1)

In [7]:
d1 = df[(df['score']>1) & (df['err']/df['cnt']<0.5)].sort_values(['score'], ascending=False)

In [8]:
d1[d1['word']=='股份有限公司']

,word,cnt,err,poly,flex,score
264,股份有限公司,1685,1,0.999407,1.300101,1.29933


In [9]:
t = trie.CharTrie()
for _, row in d1.iterrows():
    t[row['word'][::-1]] = row

In [10]:
xm2 = defaultdict(lambda: defaultdict(int)) # 存储每个候选词的前缀集合
for _, row in d1.iterrows():
    rword = row['word'][::-1]
    ret = t.prefixes(rword)
    parent = None
    maxlen = -1
    for k, v in ret:
        if k == rword: continue
        if len(k) > maxlen:
            maxlen = len(k)
            parent = k[::-1]
    if parent is not None:
        xm2[parent][row['word']] += row['cnt']

In [11]:
xdf = d1[d1['word'].isin(xm2)]

In [12]:
def xflex(x):
    if x['word'] not in xm2: return 1
    s = pd.Series(xm2[x['word']])
    return entropy(s / s.sum())

In [13]:
pd.options.mode.chained_assignment = None  # default='warn', SettingWithCopyWarning
xdf['xflex'] = xdf.apply(xflex, axis=1)

In [14]:
xdf1 = xdf[xdf['xflex']>0.1]

In [15]:
xm2['有限公司']['股份有限公司']

1685

In [16]:
xm2.keys()

dict_keys(['有限公司', '部', '专业合作社', '管理有限公司', '开发有限公司', '厂', '发展有限公司', '中心', '店', '站', '咨询有限公司', '场', '科技有限公司', '行', '商行', '设备有限公司', '工程有限公司', '服务有限公司', '处', '所', '室', '材料有限公司', '农民专业合作社', '技术有限公司', '经营部', '(上海)有限公司', '服务中心', '有限责任公司', '队', '档', '制造有限公司', '厅', '发展公司', '工程队', '种植专业合作社', '投资有限公司', '制品厂', '设备厂', '设计有限公司', '养殖专业合作社', '设备经营部', '维修部', '工作室', '百货店'])

In [17]:
[row['word'] for _, row in xdf1.iterrows() if not row['word'].endswith('有限公司')]

['店',
 '厂',
 '有限责任公司',
 '经营部',
 '中心',
 '专业合作社',
 '场',
 '站',
 '部',
 '处',
 '商行',
 '室',
 '农民专业合作社']

In [18]:
company_type = set([
    "有限公司",
    "有限责任公司",
    "店",
    "厂",
    "经营部",
    "专业合作社",
])

In [19]:
gtrie = trie.CharTrie()
for word in company_type:
    gtrie[word[::-1]] = word

In [20]:
scope = defaultdict(lambda: set())
types = set()
for _, row in d1.iterrows():
    word = row['word'][::-1]
    key, _ = gtrie.longest_prefix(word)
    if row['word'] == '股份有限公司':
        print(row, key)
    if key is not None:
        if key == word: continue
        scope[row['word'].rstrip(key[::-1])].add(key[::-1])
    else:
        types.add(row['word'])                                         

word       股份有限公司
cnt          1685
err             1
poly     0.999407
flex       1.3001
score     1.29933
Name: 264, dtype: object 司公限有


In [21]:
scope_list = [w for w in scope.keys() if w[0] != "("]

In [22]:
scope_list

['贸易',
 '科技',
 '商贸',
 '种植',
 '设备',
 '工程',
 '养殖',
 '物流',
 '服务',
 '投资管理',
 '广告',
 '房地产开发',
 '加工',
 '科技发展',
 '销售',
 '文化传播',
 '运输',
 '服装',
 '咨询',
 '设计',
 '电子',
 '投资咨询',
 '技术',
 '发展',
 '电子科技',
 '开发',
 '制造',
 '生物科技',
 '信息咨询',
 '机电设备',
 '集团',
 '投资',
 '装饰工程',
 '机械',
 '咨询服务',
 '材料',
 '配件',
 '百货',
 '金属材料',
 '建筑工程',
 '网络科技',
 '制品',
 '顾问',
 '建材',
 '养殖农民',
 '农民',
 '管理',
 '安装工程',
 '维修',
 '实业',
 '信息技术',
 '化妆品',
 '种植农民',
 '化工',
 '五金',
 '电器',
 '股份',
 '物业服务',
 '策划',
 '科技(上海)',
 '电子商务',
 '建筑材料',
 '信息科技',
 '建设',
 '教育咨询',
 '士多',
 '企业管理咨询',
 '连锁',
 '科技开发',
 '理发',
 '物业管理',
 '百货商',
 '设备制造',
 '制衣',
 '文化传媒',
 '设计制作',
 '加盟',
 '税务师事务所',
 '食用菌种植',
 '会议服务',
 '国际贸易(上海)',
 '自来水',
 '办公家具',
 '不锈钢',
 '建设投资',
 '不锈钢制品',
 '机械设备',
 '五金塑料',
 '租赁',
 '装饰工程设计',
 '食用菌',
 '网络工程',
 '种植养殖',
 '基础工程',
 '玻璃制品',
 '办公设备',
 '再生资源',
 '润滑油',
 '机械配件',
 '人力资源',
 '投资管理咨询',
 '广告装饰',
 '装饰材料',
 '农机服务',
 '日用百货']

In [23]:
type_list = [x for x in types | company_type]

In [24]:
ext_list = [x.lstrip("有限公司") for x in type_list if x.startswith("有限公司") and x != "有限公司"]

In [25]:
ext_list += [x for x in type_list if x[0]=='(']

In [26]:
ext_list

['沈阳分公司',
 '杭州分公司',
 '大连分公司',
 '分公司',
 '成都分公司',
 '第一分公司',
 '深圳分公司',
 '(吊销)',
 '(微型企业)',
 '(普通合伙)']

In [27]:
real_type_list = [x for x in type_list if x != "." and x[0]!='(' and not (x.startswith("有限公司") and x != "有限公司")]

In [28]:
real_type_list

['中心',
 '工作室',
 '门市部',
 '有限责任公司',
 '庄',
 '服务中心',
 '加油站',
 '回收站',
 '行',
 '办事处',
 '所',
 '家庭农场',
 '点',
 '园',
 '营业厅',
 '销售中心',
 '设计工作室',
 '门市',
 '种植场',
 '棋牌室',
 '门诊部',
 '城',
 '工程队',
 '部',
 '经营部',
 '室',
 '物资贸易公司',
 '供应站',
 '处',
 '网吧',
 '摊',
 '代表处',
 '专业合作社',
 '批发部',
 '维修部',
 '摩托车维修部',
 '经销处',
 '贸易部',
 '事务所',
 '厂',
 '屋',
 '服务站',
 '服务部',
 '营业部',
 '发展总公司',
 '站',
 '工程公司',
 '猪肉档',
 '服务公司',
 '有限公司',
 '库',
 '养殖场',
 '馆',
 '院',
 '柜',
 '发展公司',
 '信息咨询部',
 '队',
 '贸易商行',
 '厅',
 '咨询服务中心',
 '销售部',
 '商行',
 '经销部',
 '经济发展公司',
 '场',
 '建筑工程队',
 '档',
 '店',
 '皮革商行',
 '开发公司']

In [29]:
pd.DataFrame(scope_list).to_csv("scope_list.txt", index=False, header=False)
pd.DataFrame(real_type_list).to_csv("real_type_list.txt", index=False, header=False)
pd.DataFrame(ext_list).to_csv("ext_list.txt", index=False, header=False)